###Assignment 17-09-24

In [0]:
dbutils.fs.cp("file:/Workspace/Shared/employee.csv", "dbfs:/FileStore/streaming/input/employee.csv") 
dbutils.fs.cp("file:/Workspace/Shared/products.json", "dbfs:/FileStore/streaming/input/products.json")
dbutils.fs.cp("file:/Workspace/Shared/new_employee.csv", "dbfs:/FileStore/streaming/input/new_employee.csv")

True

Task 1

In [0]:
csv_file = "dbfs:/FileStore/streaming/input/employee.csv"
json_file = "dbfs:/FileStore/streaming/input/products.json"

df_csv = spark.read.format("csv").option("header", "true").load(csv_file)
df_json = spark.read.json(json_file)

# 1. Create Delta Table from DataFrame
df_csv.write.format("delta").mode("overwrite").save("/delta/employees_df")

# 2. Use SQL to create Delta Table
df_csv.createOrReplaceTempView("employees_view")
spark.sql("""
    CREATE TABLE IF NOT EXISTS employees_sql
    USING delta
    AS SELECT * FROM employees_view
""")

# 3. Convert CSV and JSON to Delta format
df_csv.write.format("delta").mode("overwrite").save("/delta/employees_csv")

from pyspark.sql.types import StructType, StructField, StringType, IntegerType

schema = StructType([
    StructField("ProductID", StringType(), True),
    StructField("ProductName", StringType(), True),
    StructField("Category", StringType(), True),
    StructField("Price", IntegerType(), True)
])
df_json = spark.read.schema(schema).json(json_file)

df_json.write.format("delta").mode("overwrite").save("/delta/products_json")

Task 2

In [0]:
new_employee_csv_path ="dbfs:/FileStore/streaming/input/new_employee.csv"
new_employee_df = spark.read.format("csv").option("header", "true").load(new_employee_csv_path)

from pyspark.sql.functions import col
new_employee_df = new_employee_df.withColumn("EmployeeID", col("EmployeeID").cast("int")) \
                                 .withColumn("Salary", col("Salary").cast("int"))
from delta.tables import DeltaTable

delta_table = DeltaTable.forPath(spark, "/delta/employees_csv")

delta_table.alias("old").merge(
    new_employee_df.alias("new"),
    "old.EmployeeID = new.EmployeeID"
).whenMatchedUpdate(set = {
    "EmployeeName": "new.EmployeeName",
    "Department": "new.Department",
    "JoiningDate": "new.JoiningDate",
    "Salary": "new.Salary"  
}).whenNotMatchedInsertAll().execute() 


Task 3

In [0]:
# Check transaction history
spark.sql("DESCRIBE HISTORY '/delta/employees_csv'").show()

# Perform time travel
df_time_travel = spark.read.format("delta").option("versionAsOf", 1).load("/delta/employees_csv")
df_time_travel.show()


+-------+-------------------+----------------+--------------------+---------+--------------------+----+-----------------+--------------------+-----------+-----------------+-------------+--------------------+------------+--------------------+
|version|          timestamp|          userId|            userName|operation| operationParameters| job|         notebook|           clusterId|readVersion|   isolationLevel|isBlindAppend|    operationMetrics|userMetadata|          engineInfo|
+-------+-------------------+----------------+--------------------+---------+--------------------+----+-----------------+--------------------+-----------+-----------------+-------------+--------------------+------------+--------------------+
|      3|2024-09-17 04:08:51|2929584751483774|azuser2141_mml.lo...| OPTIMIZE|{predicate -> [],...|NULL|{418182220691607}|0912-053226-p5usobai|          2|SnapshotIsolation|        false|{numRemovedFiles ...|        NULL|Databricks-Runtim...|
|      2|2024-09-17 04:08:48|292

Task 4

In [0]:
spark.sql("OPTIMIZE '/delta/employees_csv' ZORDER BY Department")

DataFrame[path: string, metrics: struct<numFilesAdded:bigint,numFilesRemoved:bigint,filesAdded:struct<min:bigint,max:bigint,avg:double,totalFiles:bigint,totalSize:bigint>,filesRemoved:struct<min:bigint,max:bigint,avg:double,totalFiles:bigint,totalSize:bigint>,partitionsOptimized:bigint,zOrderStats:struct<strategyName:string,inputCubeFiles:struct<num:bigint,size:bigint>,inputOtherFiles:struct<num:bigint,size:bigint>,inputNumCubes:bigint,mergedFiles:struct<num:bigint,size:bigint>,numOutputCubes:bigint,mergedNumCubes:bigint>,clusteringStats:struct<inputZCubeFiles:struct<numFiles:bigint,size:bigint>,inputOtherFiles:struct<numFiles:bigint,size:bigint>,inputNumZCubes:bigint,mergedFiles:struct<numFiles:bigint,size:bigint>,numOutputZCubes:bigint>,numBins:bigint,numBatches:bigint,totalConsideredFiles:bigint,totalFilesSkipped:bigint,preserveInsertionOrder:boolean,numFilesSkippedToReduceWriteAmplification:bigint,numBytesSkippedToReduceWriteAmplification:bigint,startTimeMs:bigint,endTimeMs:bigint,

Task 5

In [0]:
df_time_travel = spark.read.format("delta").option("versionAsOf", 0).load("/delta/employees_csv")
df_time_travel.show()

+----------+------------+-----------+-----------+------+
|EmployeeID|EmployeeName| Department|JoiningDate|Salary|
+----------+------------+-----------+-----------+------+
|       101|        John|         HR| 2023-01-10| 50000|
|       102|       Alice|    Finance| 2023-02-15| 70000|
|       103|        Mark|Engineering| 2023-03-20| 85000|
|       104|        Emma|      Sales| 2023-04-01| 55000|
|       105|        Liam|  Marketing| 2023-05-12| 60000|
+----------+------------+-----------+-----------+------+



Task 6

In [0]:
spark.sql("VACUUM '/delta/employees_csv' RETAIN 168 HOURS")

DataFrame[path: string]

### Structured Streaming and Transformations on Streams

In [0]:
dbutils.fs.cp("file:/Workspace/Shared/transactions.csv", "dbfs:/FileStore/streaming/input/transactions.csv") 

True

Task 1

In [0]:
static_df = spark.read.format("csv").option("header", "true").load("dbfs:/FileStore/streaming/input/transactions.csv")
schema = static_df.schema

streaming_df = spark.readStream.format("csv").option("header", "true").schema(schema).load("dbfs:/FileStore/streaming/input/")

query = streaming_df.writeStream.format("console").start()

Task 2

In [0]:
#Add a new column for the TotalAmount ( Quantity * Price ).
#Filter records where the Quantity is greater than 1.
transformed_df = streaming_df.withColumn("TotalAmount", streaming_df["Quantity"] * streaming_df["Price"]).filter(streaming_df["Quantity"] > 1)

query = transformed_df.writeStream.format("memory").queryName("transformed_stream").start()


Task 3

In [0]:
from pyspark.sql.functions import col, sum
#Group the data by ProductID and calculate the total sales for each product
aggregated_df = streaming_df.groupBy("ProductID").agg(sum(col("Quantity") * col("Price")).alias("TotalSales"))
query = aggregated_df.writeStream.format("console").outputMode("update").start()

Task 4

In [0]:
query = transformed_df.writeStream.format("parquet").option("path", "/dbfs/FileStore/parquet") \
                                   .option("checkpointLocation", "/dbfs/FileStore/checkpoint") \
                                   .start()


Task 5

In [0]:
from pyspark.sql.functions import col, to_timestamp

streaming_df = streaming_df.withColumn("TransactionDate", to_timestamp(col("TransactionDate")))

watermarked_df = streaming_df.withWatermark("TransactionDate", "1 day")

watermarked_query = watermarked_df.writeStream.format("console").start()

Task 6

In [0]:
# Stream 1: Incoming transaction data (CSV)
transactions_stream = spark.readStream.format("csv") \
    .option("header", "true") \
    .option("basePath", "dbfs:/FileStore/streaming/input/") \
    .schema("TransactionID STRING, TransactionDate DATE, ProductID STRING, Quantity INT, Price DOUBLE") \
    .load("dbfs:/FileStore/streaming/input/")

# Stream 2: Product information (JSON)
products_stream = spark.readStream.format("json") \
    .option("basePath", "dbfs:/FileStore/streaming/input/") \
    .schema("ProductID STRING, ProductName STRING, Category STRING") \
    .load("dbfs:/FileStore/streaming/input/")

# Join both streams on ProductID
joined_stream = transactions_stream.join(products_stream, "ProductID")

# Write the joined stream to the console to visualize results
query = joined_stream.writeStream \
    .format("console") \
    .outputMode("append") \
    .start()

Task 7

### Creating a Complete ETL Pipeline using Delta Live Tables (DLT)

In [0]:
dbutils.fs.cp("file:/Workspace/Shared/orders.csv", "dbfs:/FileStore/streaming/input/orders.csv") 

True

Task 1

In [0]:
import dlt
from pyspark.sql.functions import col

@dlt.table
def transformed_orders():
    # Read CSV source
    df = spark.read.csv("dbfs:/FileStore/streaming/input/orders.csv", header=True)
    
    # Transform data
    df_transformed = df.withColumn("TotalAmount", col("Quantity") * col("Price")) \
                       .filter(col("Quantity") > 1)
    
    return df_transformed

@dlt.table
def incremental_orders():
    return dlt.read_stream("transformed_orders")

Name,Type
OrderID,string
OrderDate,string
CustomerID,string
Product,string
Quantity,string
Price,string
TotalAmount,double


Task 2

In [0]:
# Load the CSV file
df_orders = spark.read.format("csv").option("header", "true").option("inferSchema", "true").load("dbfs:/FileStore/streaming/input/orders.csv")

df_orders.write.format("delta").mode("overwrite").save("/delta/orders")


In [0]:
%sql
CREATE OR REPLACE LIVE TABLE transformed_orders AS
SELECT *, Quantity * Price AS TotalAmount
FROM delta.`/delta/orders`
WHERE Quantity > 1;

Name,Type
OrderID,int
OrderDate,date
CustomerID,string
Product,string
Quantity,int
Price,int
TotalAmount,int


Task 3

1. Read the Data from the Delta Table

In [0]:
# Reading the Delta table in PySpark
df = spark.read.format("delta").load("/delta/orders")
df.show()

+-------+----------+----------+-------+--------+-----+
|OrderID| OrderDate|CustomerID|Product|Quantity|Price|
+-------+----------+----------+-------+--------+-----+
|    101|2024-01-01|      C001| Laptop|       2| 1000|
|    102|2024-01-02|      C002|  Phone|       1|  500|
|    103|2024-01-03|      C003| Tablet|       3|  300|
|    104|2024-01-04|      C004|Monitor|       1|  150|
|    105|2024-01-05|      C005|  Mouse|       5|   20|
+-------+----------+----------+-------+--------+-----+



In [0]:
%sql
-- Reading the Delta table using SQL
SELECT * FROM delta.`/delta/orders`;

OrderID,OrderDate,CustomerID,Product,Quantity,Price
101,2024-01-01,C001,Laptop,2,1000
102,2024-01-02,C002,Phone,1,500
103,2024-01-03,C003,Tablet,3,300
104,2024-01-04,C004,Monitor,1,150
105,2024-01-05,C005,Mouse,5,20


2. Update the Price of a Product (e.g., increase Laptop price by 10%)

In [0]:
from delta.tables import DeltaTable
from pyspark.sql.functions import expr

# Load the Delta table
delta_table = DeltaTable.forPath(spark, "/delta/orders")

# Update the Price of 'Laptop' by increasing it by 10%
delta_table.update(
    condition = expr("Product = 'Laptop'"),
    set = { "Price": expr("Price * 1.1") }
)


In [0]:
%sql
-- Update product price using SQL (increase Laptop price by 10%)
UPDATE delta.`/delta/orders`
SET Price = Price * 1.1
WHERE Product = 'Laptop';


num_affected_rows
1


3. Delete Rows where Quantity is Less Than 2

In [0]:
# Delete rows where Quantity is less than 2
delta_table.delete(condition = expr("Quantity < 2"))

In [0]:
%sql
-- Delete rows where Quantity is less than 2
DELETE FROM delta.`/delta/orders`
WHERE Quantity < 2;

num_affected_rows
0


4. Insert a New Record into the Delta Table

In [0]:
from pyspark.sql.types import IntegerType, StringType, StructField, StructType

# Insert a new record into the Delta table
new_data = [(106, 'C006', 'Keyboard', 2, 50)]
columns = ["OrderID", "CustomerID", "Product", "Quantity", "Price"]

# Define the schema of the Delta table
schema = StructType([
    StructField("OrderID", IntegerType(), nullable=False),
    StructField("CustomerID", StringType(), nullable=False),
    StructField("Product", StringType(), nullable=False),
    StructField("Quantity", IntegerType(), nullable=False),
    StructField("Price", IntegerType(), nullable=False)
])

# Create a DataFrame with new data and specified schema
new_df = spark.createDataFrame(new_data, schema)

# Append the new data to the Delta table
new_df.write.format("delta").mode("append").save("/delta/orders")

In [0]:
%sql
-- Insert a new record using SQL
INSERT INTO delta.`/delta/orders`
VALUES (107, '2024-01-06', 'C006', 'Keyboard', 2, 50);


num_affected_rows,num_inserted_rows
1,1


Task 4

In [0]:
# New data representing updated orders
new_data = [(101, '2024-01-10', 'C001', 'Laptop', 2, 1200),  # Updated order for Laptop
            (106, '2024-01-12', 'C006', 'Keyboard', 3, 50)]   # New order for Keyboard

columns = ["OrderID", "OrderDate", "CustomerID", "Product", "Quantity", "Price"]

# Create a DataFrame with the new data
new_df = spark.createDataFrame(new_data, columns)

# Merge the new data into the Delta table (SCD Type 2)
delta_table.alias("old").merge(
    new_df.alias("new"),
    "old.OrderID = new.OrderID"
).whenMatchedUpdate(set={
    "OrderDate": "new.OrderDate",
    "CustomerID": "new.CustomerID",
    "Product": "new.Product",
    "Quantity": "new.Quantity",
    "Price": "new.Price"
}).whenNotMatchedInsertAll().execute()

Task 5

In [0]:
# Check the transaction history using PySpark
delta_table.history().show()
# Check file details using PySpark
spark.sql("DESCRIBE DETAIL delta.`/delta/orders`").show()

+-------+-------------------+----------------+--------------------+---------+--------------------+----+-----------------+--------------------+-----------+-----------------+-------------+--------------------+------------+--------------------+
|version|          timestamp|          userId|            userName|operation| operationParameters| job|         notebook|           clusterId|readVersion|   isolationLevel|isBlindAppend|    operationMetrics|userMetadata|          engineInfo|
+-------+-------------------+----------------+--------------------+---------+--------------------+----+-----------------+--------------------+-----------+-----------------+-------------+--------------------+------------+--------------------+
|     35|2024-09-17 05:04:39|2929584751483774|azuser2141_mml.lo...| OPTIMIZE|{predicate -> [],...|NULL|{418182220691607}|0912-053226-p5usobai|         34|SnapshotIsolation|        false|{numRemovedFiles ...|        NULL|Databricks-Runtim...|
|     34|2024-09-17 05:04:37|292

In [0]:
%sql
-- Check the transaction history using SQL
DESCRIBE HISTORY delta.`/delta/orders`;
-- Check file details using SQL
DESCRIBE DETAIL delta.`/delta/orders`;

format,id,name,description,location,createdAt,lastModified,partitionColumns,clusteringColumns,numFiles,sizeInBytes,properties,minReaderVersion,minWriterVersion,tableFeatures,statistics
delta,505a9da5-3e4f-469c-ba9c-1974afaed544,null,null,dbfs:/delta/orders,2024-09-17T04:41:18.952Z,2024-09-17T05:04:39Z,List(),List(),1,1594,Map(delta.enableDeletionVectors -> true),3,7,List(deletionVectors),"Map(numRowsDeletedByDeletionVectors -> 0, numDeletionVectors -> 0)"


Task 6

In [0]:
# Query the table as it existed at a specific version
df_version = spark.read.format("delta").option("versionAsOf", 1).load("/delta/orders")
df_version.show()

# Query the table as it existed at a specific timestamp
df_timestamp = spark.read.format("delta").option("timestampAsOf", "2024-09-17T04:41:20Z").load("/delta/orders")
df_timestamp.show()

+-------+----------+----------+-------+--------+-----+
|OrderID| OrderDate|CustomerID|Product|Quantity|Price|
+-------+----------+----------+-------+--------+-----+
|    102|2024-01-02|      C002|  Phone|       1|  500|
|    103|2024-01-03|      C003| Tablet|       3|  300|
|    104|2024-01-04|      C004|Monitor|       1|  150|
|    105|2024-01-05|      C005|  Mouse|       5|   20|
|    101|2024-01-01|      C001| Laptop|       2| 1100|
+-------+----------+----------+-------+--------+-----+

+-------+----------+----------+-------+--------+-----+
|OrderID| OrderDate|CustomerID|Product|Quantity|Price|
+-------+----------+----------+-------+--------+-----+
|    101|2024-01-01|      C001| Laptop|       2| 1000|
|    102|2024-01-02|      C002|  Phone|       1|  500|
|    103|2024-01-03|      C003| Tablet|       3|  300|
|    104|2024-01-04|      C004|Monitor|       1|  150|
|    105|2024-01-05|      C005|  Mouse|       5|   20|
+-------+----------+----------+-------+--------+-----+



Task 7

In [0]:
# Optimize the table and Z-order by Product column
spark.sql("OPTIMIZE delta.`/delta/orders` ZORDER BY (Product)")
# Vacuum the Delta table to remove old versions of the files
spark.sql("VACUUM delta.`/delta/orders` RETAIN 168 HOURS")

DataFrame[path: string]

Task 8

In [0]:
from pyspark.sql.types import StructType, StructField, StringType, IntegerType, DoubleType, DateType

# Define the schema of your Parquet file
schema = StructType([
    StructField("OrderID", StringType(), True),
    StructField("OrderDate", DateType(), True),
    StructField("CustomerID", StringType(), True),
    StructField("Product", StringType(), True),
    StructField("Quantity", IntegerType(), True),
    StructField("Price", DoubleType(), True),
])

parquet_path = "/dbfs/FileStore/"
delta_path = "/dbfs/FileStore/delta_table"

# Read Parquet data with the defined schema
df_parquet = spark.read.format("parquet").schema(schema).load(parquet_path)

# Write the data in Delta format
df_parquet.write.format("delta").mode("overwrite").save(delta_path)

# Register the Delta table
spark.sql(f"CREATE TABLE delta_table USING DELTA LOCATION '{delta_path}'")

# Query the newly converted Delta table
df_converted = spark.sql("SELECT * FROM delta_table")
df_converted.show()


+-------+---------+----------+-------+--------+-----+
|OrderID|OrderDate|CustomerID|Product|Quantity|Price|
+-------+---------+----------+-------+--------+-----+
+-------+---------+----------+-------+--------+-----+



### Creating and Scheduling a Job on Databricks using Notebooks

In [0]:
from pyspark.sql.types import StructType, StructField, StringType, DateType, IntegerType, DoubleType
from pyspark.sql.functions import col

# Define the schema
schema = StructType([
    StructField("OrderID", StringType(), True),
    StructField("OrderDate", DateType(), True),
    StructField("CustomerID", StringType(), True),
    StructField("Product", StringType(), True),
    StructField("Quantity", IntegerType(), True),
    StructField("Price", DoubleType(), True),
])

# Read CSV file into a DataFrame
csv_path = "dbfs:/FileStore/streaming/input/orders.csv"
df = spark.read.format("csv").option("header", "true").schema(schema).load(csv_path)

# Add TotalAmount column and filter rows
df_transformed = df.withColumn("TotalAmount", col("Quantity") * col("Price")) \
                    .filter(col("Quantity") > 1)

# Write the transformed data to a Delta table with schema evolution enabled
delta_path = "/dbfs/FileStore/delta_table"
df_transformed.write.format("delta").option("mergeSchema", "true").mode("overwrite").save(delta_path)

# Register the Delta table
spark.sql(f"CREATE TABLE IF NOT EXISTS delta_table USING DELTA LOCATION '{delta_path}'")

# Query the Delta table to verify the transformation
df_converted = spark.sql("SELECT * FROM delta_table")
df_converted.show()

+-------+----------+----------+-------+--------+------+-----------+
|OrderID| OrderDate|CustomerID|Product|Quantity| Price|TotalAmount|
+-------+----------+----------+-------+--------+------+-----------+
|    101|2024-01-01|      C001| Laptop|       2|1000.0|     2000.0|
|    103|2024-01-03|      C003| Tablet|       3| 300.0|      900.0|
|    105|2024-01-05|      C005|  Mouse|       5|  20.0|      100.0|
+-------+----------+----------+-------+--------+------+-----------+

